# Tensorflow basic ranking

https://www.tensorflow.org/recommenders/examples/basic_ranking

In [3]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_recommenders as tfrs

## Load data

In [4]:
ratings = tfds.load("movielens/100k-ratings", split="train")

ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"]
})

In [22]:
list(ratings.take(5).as_numpy_iterator())

[{'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
  'user_id': b'138',
  'user_rating': 4.0},
 {'movie_title': b'Strictly Ballroom (1992)',
  'user_id': b'92',
  'user_rating': 2.0},
 {'movie_title': b'Very Brady Sequel, A (1996)',
  'user_id': b'301',
  'user_rating': 4.0},
 {'movie_title': b'Pulp Fiction (1994)', 'user_id': b'60', 'user_rating': 4.0},
 {'movie_title': b'Scream 2 (1997)', 'user_id': b'197', 'user_rating': 3.0}]

## Train-test split

In [6]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

movie_titles = ratings.batch(1_000_000).map(lambda x: x["movie_title"])
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [7]:
class RankingModel(tf.keras.Model):

    def __init__(self):
        super().__init__()
        embedding_dimension = 32

        # Compute embeddings for users.
        self.user_embeddings = tf.keras.Sequential([
          tf.keras.layers.experimental.preprocessing.StringLookup(
            vocabulary=unique_user_ids, mask_token=None),
          tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
        ])

        # Compute embeddings for movies.
        self.movie_embeddings = tf.keras.Sequential([
          tf.keras.layers.experimental.preprocessing.StringLookup(
            vocabulary=unique_movie_titles, mask_token=None),
          tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
        ])

        # Compute predictions.
        self.ratings = tf.keras.Sequential([
          # Learn multiple dense layers.
          tf.keras.layers.Dense(256, activation="relu"),
          tf.keras.layers.Dense(64, activation="relu"),
          # Make rating predictions in the final layer.
          tf.keras.layers.Dense(1)
        ])

    def call(self, user_id, movie_title):

        user_embedding = self.user_embeddings(user_id)
        movie_embedding = self.movie_embeddings(movie_title)

        return self.ratings(tf.concat([user_embedding, movie_embedding], axis=1))

In [24]:
RankingModel()(["42"], ["One Flew Over the Cuckoo's Nest (1975)"])

Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


Consider rewriting this model with the Functional API.


<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.01337212]], dtype=float32)>

In [13]:
task = tfrs.tasks.Ranking(
  loss = tf.keras.losses.MeanSquaredError(),
  metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

In [14]:
class MovielensModel(tfrs.models.Model):

    def __init__(self):
        super().__init__()
        self.ranking_model: tf.keras.Model = RankingModel()
        self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
          loss = tf.keras.losses.MeanSquaredError(),
          metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        rating_predictions = self.ranking_model(
            features["user_id"], features["movie_title"])

        # The task computes the loss and the metrics.
        return self.task(labels=features["user_rating"], predictions=rating_predictions)

## Fit and evaluate

In [17]:
model = MovielensModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [18]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [19]:
model.fit(cached_train, epochs=3)

Epoch 1/3
10/10 [==============================] - 0s 37ms/step - root_mean_squared_error: 2.0902 - loss: 4.0513 - regularization_loss: 0.0000e+00 - total_loss: 4.0513
Epoch 2/3
10/10 [==============================] - 0s 17ms/step - root_mean_squared_error: 1.1721 - loss: 1.3648 - regularization_loss: 0.0000e+00 - total_loss: 1.3648
Epoch 3/3
10/10 [==============================] - 0s 17ms/step - root_mean_squared_error: 1.1193 - loss: 1.2495 - regularization_loss: 0.0000e+00 - total_loss: 1.2495


In [20]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 0s 9ms/step - root_mean_squared_error: 1.1058 - loss: 1.2175 - regularization_loss: 0.0000e+00 - total_loss: 1.2175


{'root_mean_squared_error': 1.105752944946289,
 'loss': 1.1949037313461304,
 'regularization_loss': 0,
 'total_loss': 1.1949037313461304}

## Next steps

In most cases, a ranking model can be substantially improved by using more features rather than just user and candidate identifiers. To see how to do that, have a look at the side features tutorial.

More steps needed to convert this into a practical solutions.